# Iterating over multiple refutation tests
The objective of this notebook is to compare the ability of refuters to detect the problems in a given set of estimators.
Note:
This notebook makes use of the optional dependencies:
- pygraphviz
- causalml

## Import Dependencies

In [1]:
from dowhy.datasets import linear_dataset
from dowhy import CausalModel
import causalml

## Inspection Parameters
These parameters give us the option of inspecting the intermediate steps to sanity check the steps performed

In [2]:
inspect_datasets = True
inspect_models = True
inspect_identified_estimands = True
inspect_estimates = True
inspect_refutations = True

## Estimator List
We pass a list of strings, corresponding to the estimators of interest

In [3]:
estimator_list = ["backdoor.propensity_score_matching", "backdoor.propensity_score_weighting", "backdoor.causalml.inference.meta.LRSRegressor"]
method_params= [ None, None, { "init_params":{} } ]

## Refuter List
A list of strings, corresponding to each refuter we wish to run

In [4]:
refuter_list = ["bootstrap_refuter", "data_subset_refuter"]

## Create the Datasets

In [5]:
# Parameters for creating the Dataset
TREATMENT_IS_BINARY = True
BETA = 10
NUM_SAMPLES = 5000
NUM_CONFOUNDERS = 5
NUM_INSTRUMENTS = 3
NUM_EFFECT_MODIFIERS = 2

# Creating a Linear Dataset with the given parameters
linear_data = linear_dataset(
            beta = BETA,
            num_common_causes = NUM_CONFOUNDERS,
            num_instruments = NUM_INSTRUMENTS,
            num_effect_modifiers = NUM_EFFECT_MODIFIERS,
            num_samples = NUM_SAMPLES,
            treatment_is_binary = True
        )
# Other datasets come here 


# Append them together in an array
datasets = [linear_data]


## Inspect Data

In [6]:
dataset_num = 1
if inspect_datasets is True:
    for data in datasets:
        print("####### Dataset {}###########################################################################################".format(dataset_num))
        print(data['df'].head())
        print("#############################################################################################################")
        dataset_num += 1

####### Dataset 1###########################################################################################
         X0        X1   Z0        Z1   Z2        W0        W1        W2  \
0 -1.372310 -0.633368  0.0  0.353144  0.0 -0.089772 -0.838580  0.284335   
1  0.713931  0.802709  0.0  0.653470  1.0 -1.083905 -1.160895  0.591947   
2 -0.384362 -0.451062  0.0  0.341027  1.0 -1.344159 -0.966449 -0.181318   
3  1.515319 -1.363540  0.0  0.835704  1.0  0.342380 -1.004874  1.552707   
4  0.368875 -0.829938  0.0  0.071521  0.0 -0.128446 -1.994744  0.519826   

         W3        W4     v0          y  
0  1.223258 -1.124599   True   2.034000  
1  1.779737 -0.612423   True  11.125450  
2 -0.290884 -1.743305  False  -9.777346  
3  1.042000 -2.330312   True  10.343818  
4  0.479455 -1.345510  False  -4.253140  
#############################################################################################################


## Create the CausalModels

In [7]:
models = []
for data in datasets:
    model = CausalModel(
                data = data['df'],
                treatment = data['treatment_name'],
                outcome = data['outcome_name'],
                graph = data['gml_graph']
            )
    models.append(model)

INFO:dowhy.causal_model:Model to find the causal effect of treatment ['v0'] on outcome ['y']


## Inspect Models

In [8]:
model_num = 1
if inspect_models is True:
    for model in models:
        print("####### Model {}#############################################################################################".format(model_num))
        print("Common Causes:",model._common_causes)
        print("Effect Modifiers:",model._effect_modifiers)
        print("Instruments:",model._instruments)
        print("Outcome:",model._outcome)
        print("Treatment:",model._treatment)
        print("#############################################################################################################")
        model_num += 1

####### Model 1#############################################################################################
Common Causes: ['W4', 'W2', 'W1', 'W3', 'W0', 'Unobserved Confounders']
Effect Modifiers: ['X0', 'X1']
Instruments: ['Z2', 'Z1', 'Z0']
Outcome: ['y']
Treatment: ['v0']
#############################################################################################################


## Identify Effect

In [9]:
identified_estimands = []
for model in models:
    identified_estimand = model.identify_effect(proceed_when_unidentifiable=True)
    identified_estimands.append(identified_estimand)

INFO:dowhy.causal_identifier:Continuing by ignoring these unobserved confounders because proceed_when_unidentifiable flag is True.
INFO:dowhy.causal_identifier:Instrumental variables for treatment and outcome:['Z2', 'Z1', 'Z0']
INFO:dowhy.causal_identifier:Frontdoor variables for treatment and outcome:[]


## Identified Estimands

In [10]:
estimand_count = 1
for estimand in identified_estimands:
    print("####### Identified Estimand {}#####################################################################################".format(estimand_count))
    print(estimand)
    print("###################################################################################################################")
    estimand_count += 1

####### Identified Estimand 1#####################################################################################
Estimand type: nonparametric-ate

### Estimand : 1
Estimand name: backdoor1
Estimand expression:
  d                                 
─────(Expectation(y|W4,W2,W1,W3,W0))
d[v₀]                               
Estimand assumption 1, Unconfoundedness: If U→{v0} and U→y then P(y|v0,W4,W2,W1,W3,W0,U) = P(y|v0,W4,W2,W1,W3,W0)

### Estimand : 2
Estimand name: backdoor2
Estimand expression:
  d                                    
─────(Expectation(y|W4,W2,W1,W3,W0,X1))
d[v₀]                                  
Estimand assumption 1, Unconfoundedness: If U→{v0} and U→y then P(y|v0,W4,W2,W1,W3,W0,X1,U) = P(y|v0,W4,W2,W1,W3,W0,X1)

### Estimand : 3
Estimand name: backdoor3
Estimand expression:
  d                                    
─────(Expectation(y|W4,W2,W1,W3,W0,X0))
d[v₀]                                  
Estimand assumption 1, Unconfoundedness: If U→{v0} and U→y then P(y|v0,W4,W

## Estimate Effect

In [11]:
estimate_list = []
for i in range(len(identified_estimands)):
    for j in range(len(estimator_list)):
        estimate = model.estimate_effect(
                        identified_estimands[i],
                        method_name=estimator_list[j],
                        method_params=method_params[j]
                  )
        estimate_list.append(estimate)

INFO:dowhy.causal_estimator:INFO: Using Propensity Score Matching Estimator
INFO:dowhy.causal_estimator:b: y~v0+W4+W2+W1+W3+W0+X1+X0
/home/amit/py-envs/env3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Weighting Estimator
INFO:dowhy.causal_estimator:b: y~v0+W4+W2+W1+W3+W0+X1+X0
/home/amit/py-envs/env3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot 

{'X':            W4        W2        W1        W3        W0        X1        X0  \
0   -1.124599  0.284335 -0.838580  1.223258 -0.089772 -0.633368 -1.372310   
1   -0.612423  0.591947 -1.160895  1.779737 -1.083905  0.802709  0.713931   
2   -1.743305 -0.181318 -0.966449 -0.290884 -1.344159 -0.451062 -0.384362   
3   -2.330312  1.552707 -1.004874  1.042000  0.342380 -1.363540  1.515319   
4   -1.345510  0.519826 -1.994744  0.479455 -0.128446 -0.829938  0.368875   
..        ...       ...       ...       ...       ...       ...       ...   
995 -2.515948  0.660984 -1.169142 -2.272146 -1.210642 -0.284190 -0.127452   
996 -2.943305  1.765768 -0.453093  1.811460  1.207236  1.056658  2.059214   
997 -0.903931 -0.733567 -0.631920  2.074003 -1.104834 -0.227741 -0.262214   
998 -0.107884  0.866378 -1.644002 -0.046340 -1.318515  0.888535  1.662692   
999 -1.664178  0.364437 -0.018848  0.440983 -1.575637 -0.289049 -0.646924   

           X0        X1  
0   -1.372310 -0.633368  
1    0.713931  0.

## Estimate Values

In [12]:
estimand_count = 1
if inspect_estimates is True:
    for estimand in estimate_list:
        print("####### Estimand {}#######################################################################################".format(estimand_count))
        print("*** Class Name ***")
        print()
        print(estimand.params['estimator_class'])
        print()
        print(estimand)
        print("########################################################################################################")
        print()
        estimand_count += 1
    

####### Estimand 1#######################################################################################
*** Class Name ***

<class 'dowhy.causal_estimators.propensity_score_matching_estimator.PropensityScoreMatchingEstimator'>

*** Causal Estimate ***

## Identified estimand
Estimand type: nonparametric-ate

## Realized estimand
b: y~v0+W4+W2+W1+W3+W0+X1+X0
Target units: ate

## Estimate
Mean value: 12.89367255934708

########################################################################################################

####### Estimand 2#######################################################################################
*** Class Name ***

<class 'dowhy.causal_estimators.propensity_score_weighting_estimator.PropensityScoreWeightingEstimator'>

*** Causal Estimate ***

## Identified estimand
Estimand type: nonparametric-ate

## Realized estimand
b: y~v0+W4+W2+W1+W3+W0+X1+X0
Target units: ate

## Estimate
Mean value: 12.075122341243496

###########################################

## Refute Estimate

In [13]:
refutation_list = []
for estimand in identified_estimands:
    for estimate in estimate_list: 
        for refuter in refuter_list:
            ref = model.refute_estimate(estimand, estimate,method_name=refuter)
            refutation_list.append(ref)

INFO:dowhy.causal_refuters.bootstrap_refuter:All variables required: Running bootstrap adding noise to confounders, instrumental variables and effect modifiers.
INFO:dowhy.causal_refuters.bootstrap_refuter:INFO: The chosen variables are: W4,W2,W1,W3,W0,X1,X0,Z2,Z1,Z0,X0,X1
INFO:dowhy.causal_refuters.bootstrap_refuter:Refutation over 100 simulated datasets of size 1000 each
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Matching Estimator
INFO:dowhy.causal_estimator:b: y~v0+W4+W2+W1+W3+W0+X1+X0
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Matching Estimator
INFO:dowhy.causal_estimator:b: y~v0+W4+W2+W1+W3+W0+X1+X0
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Matching Estimator
INFO:dowhy.causa

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Matching Estimator
INFO:dowhy.causal_estimator:b: y~v0+W4+W2+W1+W3+W0+X1+X0
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Matching Estimator
INFO:dowhy.causal_estimator:b: y~v0+W4+W2+W1+W3+W0+X1+X0
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Matching Estimator
INFO:dowhy.causal_estimator:b: y~v0+W4+W2+W1+W3+W0+X1+X0
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Matching Est

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Matching Estimator
INFO:dowhy.causal_estimator:b: y~v0+W4+W2+W1+W3+W0+X1+X0
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Matching Estimator
INFO:dowhy.causal_estimator:b: y~v0+W4+W2+W1+W3+W0+X1+X0
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Matching Estimator
INFO:dowhy.causal_estimator:b: y~v0+W4+W2+W1+W3+W0+X1+X0
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Matching Est

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Matching Estimator
INFO:dowhy.causal_estimator:b: y~v0+W4+W2+W1+W3+W0+X1+X0
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Matching Estimator
INFO:dowhy.causal_estimator:b: y~v0+W4+W2+W1+W3+W0+X1+X0
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Matching Estimator
INFO:dowhy.causal_estimator:b: y~v0+W4+W2+W1+W3+W0+X1+X0
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Matching Est

INFO:dowhy.causal_estimator:b: y~v0+W4+W2+W1+W3+W0+X1+X0
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Matching Estimator
INFO:dowhy.causal_estimator:b: y~v0+W4+W2+W1+W3+W0+X1+X0
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Matching Estimator
INFO:dowhy.causal_estimator:b: y~v0+W4+W2+W1+W3+W0+X1+X0
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Matching Estimator
INFO:dowhy.causal_estimator:b: y~v0+W4+W2+W1+W3+W0+X1+X0
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
INFO:dowhy.c

...

INFO:causalml:    Gini (Treatment):     0.9954
INFO:dowhy.causal_estimator:INFO: Using CausalML Estimator
INFO:dowhy.causal_estimator:b: y~v0+W4+W2+W1+W3+W0+X1+X0
INFO:causalml:Error metrics for group True
INFO:causalml:    RMSE   (Control):     3.0193
INFO:causalml:    RMSE (Treatment):     0.7014
INFO:causalml:   sMAPE   (Control):     0.5388
INFO:causalml:   sMAPE (Treatment):     0.1490
INFO:causalml:    Gini   (Control):     0.7211
INFO:causalml:    Gini (Treatment):     0.9949
INFO:dowhy.causal_estimator:INFO: Using CausalML Estimator
INFO:dowhy.causal_estimator:b: y~v0+W4+W2+W1+W3+W0+X1+X0


{'X':            W4        W2        W1        W3        W0        X1        X0  \
199 -0.887451 -0.781149 -0.243760 -0.813839 -0.018489 -1.809168  1.396317   
249 -0.735623  0.814306 -1.693160  1.847391 -0.791449 -0.970486 -1.068904   
75  -0.681840 -0.860902 -0.003540  0.004867 -0.711607  1.445133  2.210988   
629 -1.087554  1.775480  1.171119  0.254408  0.110085 -0.901117  0.490717   
719 -1.646363  0.269526 -1.677945  1.596705 -1.646524  0.649971 -0.577936   
..        ...       ...       ...       ...       ...       ...       ...   
503 -2.066140  2.033010 -1.559405  0.352297  0.418520 -0.333911  1.084480   
123 -0.755355  1.402171 -0.488604  1.421476 -2.454651  1.055428 -1.229143   
425 -0.547299  1.010457  0.355019  0.509403 -0.299352 -1.751637  1.623652   
153 -0.787940  1.279546 -1.034601  0.763826 -0.810432 -1.157911 -0.536019   
180 -0.772968  0.508917 -1.168432 -0.936875  1.844232 -0.008911  0.298550   

           X0        X1  
199  1.396317 -1.809168  
249 -1.068904 -0.

INFO:causalml:Error metrics for group True
INFO:causalml:    RMSE   (Control):     3.0336
INFO:causalml:    RMSE (Treatment):     0.7137
INFO:causalml:   sMAPE   (Control):     0.5254
INFO:causalml:   sMAPE (Treatment):     0.1430
INFO:causalml:    Gini   (Control):     0.7298
INFO:causalml:    Gini (Treatment):     0.9946
INFO:dowhy.causal_estimator:INFO: Using CausalML Estimator
INFO:dowhy.causal_estimator:b: y~v0+W4+W2+W1+W3+W0+X1+X0
INFO:causalml:Error metrics for group True
INFO:causalml:    RMSE   (Control):     2.9814
INFO:causalml:    RMSE (Treatment):     0.6610
INFO:causalml:   sMAPE   (Control):     0.5544
INFO:causalml:   sMAPE (Treatment):     0.1489
INFO:causalml:    Gini   (Control):     0.6981


{'X':            W4        W2        W1        W3        W0        X1        X0  \
772 -1.631390  0.465915 -1.149010  1.590925 -0.376748 -1.413846 -0.295266   
75  -0.681840 -0.860902 -0.003540  0.004867 -0.711607  1.445133  2.210988   
897 -0.501089 -1.031997 -1.334448 -0.012652 -0.451207 -0.315850 -0.369898   
197 -1.572938  0.609204 -2.470786  2.406344 -0.928691 -0.722715  1.007663   
650 -1.089175  0.031213 -2.627317  1.213359  0.277555 -1.571450  0.129622   
..        ...       ...       ...       ...       ...       ...       ...   
112  0.060647  0.642094  1.051395  1.234630 -0.601046  0.340267  2.262831   
818 -2.294772  1.081263 -0.110948  0.240523 -2.452061 -0.676310  1.024324   
646 -0.626251  1.021268  0.186793  1.691995 -1.534219 -0.559809  0.720521   
720 -0.178107  0.545358 -1.188979 -2.009287 -1.712708  0.183073  2.293672   
593 -0.197920  1.086551 -0.777233 -1.511006 -0.413144  0.015110  0.344828   

           X0        X1  
772 -0.295266 -1.413846  
75   2.210988  1.

INFO:causalml:    Gini (Treatment):     0.9955
INFO:dowhy.causal_refuters.data_subset_refuter:Making use of Bootstrap as we have more than 100 examples.
                 Note: The greater the number of examples, the more accurate are the confidence estimates


## Refutation Values

In [14]:
refuter_count = 1
if inspect_refutations is True:
    for refutation in refutation_list:
        print("####### Refutation {}#######################################################################################".format(refuter_count))
        print("*** Class Name ***")
        print()
        print(refutation.refutation_type)
        print()
        print(refutation)
        print("########################################################################################################")
        print()
        refuter_count += 1

####### Refutation 1#######################################################################################
*** Class Name ***

Refute: Bootstrap Sample Dataset

Refute: Bootstrap Sample Dataset
Estimated effect:12.89367255934708
New effect:12.374411274559812
p value:0.28

########################################################################################################

####### Refutation 2#######################################################################################
*** Class Name ***

Refute: Use a subset of data

Refute: Use a subset of data
Estimated effect:12.89367255934708
New effect:12.616442370428834
p value:0.31000000000000005

########################################################################################################

####### Refutation 3#######################################################################################
*** Class Name ***

Refute: Bootstrap Sample Dataset

Refute: Bootstrap Sample Dataset
Estimated effect:12.075122341243496
Ne